In [1]:
# import pickle
import pandas as pd
import os

In [2]:
%%capture
%load_ext kedro.ipython

### Create 'User Annotation (to combine)' sheet

In [41]:
pred_cluster = catalog.load("final_predicted_cluster")
filtered_data = catalog.load("filtered_data_with_keywords")

[08/23/24 11:59:06] INFO     Loading data from final_predicted_cluster (CSVDataset)...          ]8;id=358322;file://c:\Users\Joycelyn\anaconda3\envs\genai-hh\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=58659;file://c:\Users\Joycelyn\anaconda3\envs\genai-hh\Lib\site-packages\kedro\io\data_catalog.py#508\508]8;;\

                    INFO     Loading data from filtered_data_with_keywords (ParquetDataset)...  ]8;id=735584;file://c:\Users\Joycelyn\anaconda3\envs\genai-hh\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=4911;file://c:\Users\Joycelyn\anaconda3\envs\genai-hh\Lib\site-packages\kedro\io\data_catalog.py#508\508]8;;\

In [42]:
pred_cluster = pred_cluster.drop(columns=["body_content","cluster"],axis=1)

merged_df = pd.merge(
    pred_cluster,
    filtered_data[["id","content_category","page_views","article_category_names"]],
    left_on='id',
    right_on="id",
    how='left'
)

merged_df["Action"] = ""
merged_df["Subgroup"] = ""
merged_df["Main Article Structure? (Y)"] = ""
merged_df["Main Article URL? (Y)"] = ""
merged_df["group_subgroup"] = ""
merged_df["Status"] = ""
merged_df["Optional: additional content to add for harmonisation"] = ""

In [43]:
merged_df = merged_df.rename(columns={
    "id": "article_id", 
    "new_cluster": "group_number",
    "cluster_kws": "group_description"
})

cols_order = ["group_number","Action", "Subgroup", "Main Article Structure? (Y)", "Main Article URL? (Y)", "group_subgroup", "Status", "Optional: additional content to add for harmonisation", "group_description", "article_id", "title", "url", "content_category", "article_category_names", "page_views"]

merged_df = merged_df.loc[:,cols_order]
merged_df = merged_df.dropna(subset=['group_description'])
merged_df

,group_number,Action,Subgroup,Main Article Structure? (Y),Main Article URL? (Y),group_subgroup,Status,Optional: additional content to add for harmonisation,group_description,article_id,title,url,content_category,article_category_names,page_views
0,2,,,,,,,,"['pneumonia', 'pneumococcal', 'disease', 'lung...",1437303,Pneumonia,https://www.healthhub.sg/a-z/diseases-and-cond...,diseases-and-conditions,"Conditions and Illnesses,",3536
1,2,,,,,,,,"['pneumonia', 'pneumococcal', 'disease', 'lung...",1437301,Pneumococcal Disease,https://www.healthhub.sg/a-z/diseases-and-cond...,diseases-and-conditions,"Conditions and Illnesses,",1171
3,6,,,,,,,,"['gastroenteritis', 'poisoning', 'diarrhoea', ...",1437734,Stomach Flu in Children,https://www.healthhub.sg/a-z/diseases-and-cond...,diseases-and-conditions,"Child and Teen Health,",2316
4,6,,,,,,,,"['gastroenteritis', 'poisoning', 'diarrhoea', ...",1437507,Food Poisoning in Children,https://www.healthhub.sg/a-z/diseases-and-cond...,diseases-and-conditions,"Food & Nutrition,",902
5,6,,,,,,,,"['gastroenteritis', 'poisoning', 'diarrhoea', ...",1437726,Common childhood conditions – Gastroenteritis,https://www.healthhub.sg/a-z/diseases-and-cond...,diseases-and-conditions,"Conditions and Illnesses,",138
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
663,666,,,,,,,,"['breast', 'cancer', 'mammogram', 'screening',...",1444590,5 Ways to Psych Yourself for a Mammogram,https://www.healthhub.sg/live-healthy/5-ways-t...,live-healthy-articles,"Conditions and Illnesses,",1184
664,666,,,,,,,,"['breast', 'cancer', 'mammogram', 'screening',...",1435040,Breast Screening Subsidies in Singapore,https://www.healthhub.sg/a-z/costs-and-financi...,cost-and-financing,"Conditions and Illnesses,",10855
665,666,,,,,,,,"['breast', 'cancer', 'mammogram', 'screening',...",1437337,Breast cancer,https://www.healthhub.sg/a-z/diseases-and-cond...,diseases-and-conditions,"Conditions and Illnesses,",6579
666,667,,,,,,,,"['hepatitis', 'liver', 'infected', 'hav', 'vir...",1437477,Hepatitis B,https://www.healthhub.sg/a-z/diseases-and-cond...,diseases-and-conditions,"Conditions and Illnesses,",5330


### Create 'Progress Overview' sheet

In [44]:
grouped_df = merged_df.groupby('group_number').agg({
    'title': lambda x: x.count(),
}).reset_index()
grouped_df = grouped_df.rename(columns={"title": "number_of_articles_in_group"})
grouped_df["number reviewed"] = ""
grouped_df["Progress"] = ""
grouped_df

,group_number,number_of_articles_in_group,number reviewed,Progress
0,2,2,,
1,6,3,,
2,17,2,,
3,38,2,,
4,64,3,,
...,...,...,...,...
81,712,16,,
82,716,11,,
83,717,6,,
84,732,7,,


### Export file

In [45]:
folder_path = r"..\data\08_reporting"
save_path = os.path.join(folder_path,"user_annotation_combine.xlsx")

with pd.ExcelWriter(save_path) as writer:
    merged_df.to_excel(writer, sheet_name='User Annotation (to combine)', index=False)
    grouped_df.to_excel(writer, sheet_name='Progress Overview', index=False)